In [10]:
import torch
import os
import sys
sys.path.append('/home/yuhaowang/project/FMBC/dino_stage2')
import vision_transformer as vits

In [8]:
def load_pretrained_weights(model, pretrained_weights, checkpoint_key):
    if os.path.isfile(pretrained_weights):
        state_dict = torch.load(pretrained_weights, map_location="cpu")
        if checkpoint_key is not None and checkpoint_key in state_dict:
            print(f"Take key {checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = model.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(pretrained_weights, msg))
        model.eval()
        return model

In [15]:
slide_encoder=vits.VITRMIM(vits.__dict__['vit_small'](embed_dim=384))
slide_encoder=load_pretrained_weights(slide_encoder, '/home/yuhaowang/project/FMBC/dino_stage2/output/checkpoint0020.pth', 'teacher')

/tmp/ipykernel_3431826/2337529237.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(pretrained_weights, map_location="cpu")


Take key teacher in provided checkpoint dict
Pretrained weights found at /home/yuhaowang/project/FMBC/dino_stage2/output/checkpoint0020.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])


In [13]:
slide_encoder

VisionTransformer(
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
        )
      )
      (drop_path): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
  (head): Identity()
)

In [3]:
import pandas as pd
data_dir='/home/yuhaowang/project/FMBC/finetune/dataset_csv/mutation/BRCA-6-gene_TCGA.csv'
data=pd.read_csv(data_dir)
#BRAF	EGFR	FAT1	KRAS	LRP1B	TP53	HRD
for column in ['BRAF','EGFR','FAT1','KRAS','LRP1B','TP53','HRD']:
    print(data[column].value_counts())

BRAF
False    925
True       7
Name: count, dtype: int64
EGFR
False    923
True       9
Name: count, dtype: int64
FAT1
False    904
True      28
Name: count, dtype: int64
KRAS
False    926
True       6
Name: count, dtype: int64
LRP1B
False    906
True      26
Name: count, dtype: int64
TP53
False    602
True     330
Name: count, dtype: int64
HRD
False    481
True     451
Name: count, dtype: int64
